# Tiny Dataset notebook

In [17]:
import time
import os
import openai
from freeplay import Freeplay, RecordPayload, CallInfo, ResponseInfo, SessionInfo, CallInfo
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")
OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
FREEPLAY_API_KEY = os.environ.get("FREEPLAY_API_KEY")


# create your a freeplay client object
fpClient = Freeplay(
    freeplay_api_key=os.getenv("FREEPLAY_API_KEY"),
    api_base="https://8thlight.freeplay.ai/api"
)

## PROMPT FETCH ##
# set the prompt variables
prompt_vars = {"keyA": "valueA"}
# get a formatted prompt
formatted_prompt = fpClient.prompts.get_formatted(project_id="473ef06b-2883-41da-85dd-9ccec95c95c6",
                                                  template_name="Evaluation of Summary Capabilities",
                                                  environment="latest",
                                                  variables=prompt_vars)

assistant_messages = [message for message in formatted_prompt.messages if message['role'] == "assistant"]

file_path = '/Users/kroy/ai-cohort/ai_cohort/data/ctg-studies-tiny.csv'

test_file = openai.files.create(
    file=open(file_path, "rb"),
    purpose='assistants'
)

my_assistant = openai.beta.assistants.create(
    model="gpt-4o",
    instructions=getattr(formatted_prompt, "system_content"),
    name="Data Analysis Chatbot",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
          "file_ids": [test_file.id]
        }
    }
)

my_thread = openai.beta.threads.create()

my_thread_message = openai.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content=assistant_messages[0]["content"],
  attachments=[
    {
      "file_id": test_file.id,
      "tools": [
        { "type": "code_interpreter" }
      ]
    }
  ]
)

my_run = openai.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=my_assistant.id,
)

keep_retrieving_run = openai.beta.threads.runs.retrieve(
    thread_id=my_thread.id,
    run_id=my_run.id
)

run_steps = openai.beta.threads.runs.steps.list(
  thread_id=my_thread.id,
  run_id=my_run.id
)

def pollAnswers(threadId, runId):
        completed = False
        start = time.time()
        while not completed:
            try:
                response = openai.beta.threads.runs.retrieve(
                    thread_id=threadId,
                    run_id=runId
                )

                if response.status == 'completed':
                    end = time.time()
                    print('Run completed!')
                    respmsg = openai.beta.threads.messages.list(threadId)
                    print(respmsg.data[0].content[0].text.value)
                    outgoingMessages = respmsg.data[0].content[0].text.value
                    completed = True

                    # # add the response to your message set
                    all_messages = formatted_prompt.all_messages(
                        {'role': respmsg.data[0].role,
                         'content': respmsg.data[0].content[0].text.value}
                    )
                    
                    # ## RECORD ##
                    # # create a session
                    session = fpClient.sessions.create()
                    
                    # # build the record payload
                    payload = RecordPayload(
                        all_messages=all_messages,
                        inputs=prompt_vars,
                        session_info=session,
                        prompt_info=formatted_prompt.prompt_info,
                        call_info=CallInfo.from_prompt_info(formatted_prompt.prompt_info, start_time=start, end_time=end),
                        response_info=ResponseInfo(
                            is_complete=completed
                        )
                    )
                    # record the LLM interaction
                    fpClient.recordings.create(payload)
                    
                    # bstack.emit('chat.message', {'message': outgoingMessages})
                    break

                if response.status == 'expired':
                    print('Task expired!')
                    respmsg = openai.beta.threads.messages.list(threadId)
                    print(respmsg.data[0].content[0].text.value)

                    outgoingMessages = respmsg.data[0].content[0].text.value
                    completed = True
                    # bstack.emit('chat.message', {'message': outgoingMessages})
                    break

                if response.status == 'requires_action':
                    print('Run Action Required!')

                    tool_outputs = []
                    for ra in response.required_action.submit_tool_outputs.tool_calls:
                        if ra.type == 'function' and ra.function.name in functions:
                            print(f'Running {ra.function.name} with arguments {ra.function.arguments}')
                            ret = functions[ra.function.name](ra.function.arguments)
                            print('Returned', ret)
                            tool_outputs.append({
                                'tool_call_id': ra.id,
                                'output': json.dumps(ret)
                            })
                        else:
                            tool_outputs.append({
                                'tool_call_id': ra.id,
                                'output': 'There is a problem, function not found!'
                            })

                    run = openai.beta.threads.runs.submitToolOutputs(
                        threadId,
                        runId,
                        {'tool_outputs': tool_outputs}
                    )

                if response.status in ('failed', 'cancelled', 'expired', 'cancelling'):
                    print(f'Completed {response.status}')
                    completed = True
                    break

                if response.status in ('in_progress', 'queued'):
                    print(f'Run {response.status}')
                    time.sleep(2)  # Poll every 2 seconds

            except Exception as e:
                print('Error in polling (retrying):', str(e))
                # Handle the exception or retry logic as necessary

pollAnswers(my_thread.id, my_run.id)

Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run completed!
({'study_purpose': 0.464, 'study_background': 0.464, 'demographics': 0.002, 'outcomes': 0.04, 'complications': 0.026, 'jazz_hands': 0.004}, 0.249, [[1.0, 0.465, 0.0, 0.0, 0.0, 0.0], [0.465, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]], 0.751)
